# Assignment 3, Qilin Zhou, 1/27/2024

## Question 1

### Directory Preparation


In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
%cd /content/gdrive/MyDrive/Colab\ Notebooks

/content/gdrive/MyDrive/Colab Notebooks


In [3]:
import sklearn
import myutils

OS:  Linux-6.1.58+-x86_64-with-glibc2.35
python:  3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
sklearn:  1.2.2
numpy:  1.23.5
pandas:  1.5.3
matplotlib:  3.7.1
tensorflow:  2.15.0
Sun Jan 28 04:37:22 2024


### Load the MNIST dataset

In [4]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml("mnist_784", version=1, as_frame=False)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [5]:
X, y = mnist["data"], mnist["target"]
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

* Considering execution times, I will use use 1000 training instances and 250 test instances (with 8:2 ratio) for OvO and OvR examination.

### One-Versus-the-Rest

In [6]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
svc = SVC(random_state=42)

myutils.tic()
svm_clf_ovr = OneVsRestClassifier(svc)
svm_clf_ovr.fit(X_train[:1000], y_train[:1000])
myutils.toc()

Duration = 0.74


### One-Versus-One

In [7]:
from sklearn.multiclass import OneVsOneClassifier

myutils.tic()
svm_clf_ovo = OneVsOneClassifier(svc)
svm_clf_ovo.fit(X_train[:1000], y_train[:1000])
myutils.toc()

Duration = 0.38


* One-versus-one strategy runs faster than one-versus-rest, in terms of SVM classification.
* Because the training complexity of SVM is highly dependent on the size of the training set, SVM does not perform well for large datasets. OvO is thus preferred for SVM, with smaller training sets for each classifier.

### Accuracy Comparison

In [8]:
from sklearn.metrics import accuracy_score
y_pred_ovr = svm_clf_ovr.predict(X_test[:250])
y_pred_ovo = svm_clf_ovo.predict(X_test[:250])

accuracy_ovr = accuracy_score(y_test[:250], y_pred_ovr)
accuracy_ovo = accuracy_score(y_test[:250], y_pred_ovo)

In [9]:
print(accuracy_ovr)
print(accuracy_ovo)

0.908
0.904


* Both classifiers have similar accuracy around 91%.

### Grid Search - Tuning Hyperparameters

In [47]:
from sklearn.model_selection import GridSearchCV

param_grid  = {"estimator__C": [1, 5, 10, 100], "estimator__gamma": ["scale", "auto"]}

In [48]:
svm_clf_ovo = OneVsOneClassifier(svc)
grid_search_ovo = GridSearchCV(svm_clf_ovo, param_grid,
                              cv=3, verbose=2, n_jobs=-1)

grid_search_ovo.fit(X_train[:1000], y_train[:1000])
grid_search_ovo.best_params_

Fitting 3 folds for each of 8 candidates, totalling 24 fits


{'estimator__C': 5, 'estimator__gamma': 'scale'}

In [49]:
svm_clf_ovr = OneVsRestClassifier(svc)
grid_search_ovr = GridSearchCV(svm_clf_ovr, param_grid,
                              cv=3, verbose=2, n_jobs=-1)

grid_search_ovr.fit(X_train[:1000], y_train[:1000])
grid_search_ovr.best_params_

Fitting 3 folds for each of 8 candidates, totalling 24 fits


{'estimator__C': 5, 'estimator__gamma': 'scale'}

In [50]:
best_svm_clf_ovo = grid_search_ovo.best_estimator_
best_svm_clf_ovr = grid_search_ovr.best_estimator_

In [51]:
y_pred_ovo = best_svm_clf_ovo.predict(X_test[:250])
y_pred_ovr =best_svm_clf_ovr.predict(X_test[:250])

In [52]:
accuracy_ovo = accuracy_score(y_test[:250], y_pred_ovo)
accuracy_ovr = accuracy_score(y_test[:250], y_pred_ovr)

In [53]:
print(accuracy_ovr)
print(accuracy_ovo)

0.924
0.932


* The tuned SVM classifier under both strategies has increased its accuracy, to around 93%.

## Question 2

### Data Preparation

In [36]:
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [37]:
fetch_housing_data()

In [38]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [39]:
housing = load_housing_data()

### Split into Test and Train Sets

In [40]:
import numpy as np
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

In [41]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [42]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [43]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

### Data Preprocessing

In [44]:
from sklearn.base import BaseEstimator, TransformerMixin

col_names = "total_rooms", "total_bedrooms", "population", "households"
rooms_ix, bedrooms_ix, population_ix, households_ix = [
    housing.columns.get_loc(c) for c in col_names]

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [45]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

housing_num = housing.drop("ocean_proximity", axis=1)

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing)

### Train an SVM Regressor

In [46]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

In [59]:
param_grid = [{"kernel": ["linear"], "C": [0.1, 1, 10, 100]},
              {"kernel": ["rbf"], "C":[0.1, 1, 10, 100], "gamma":[1, 10]},
]

grid_search = GridSearchCV(SVR(), param_grid, cv=5,
                           scoring="neg_mean_squared_error",
                           return_train_score=True, n_jobs=12)

# Minimize the training size due to execution time
grid_search.fit(housing_prepared[:1000], housing_labels[:1000])
grid_search.best_params_

{'C': 100, 'kernel': 'linear'}

* By grid search, we obtain the best SVR model by setting the kernel as linear with C hyperparameter as 100.

## Question 3

### Import dataset

In [78]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

### Split dataset

In [79]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### Grid Search - Tuning hyperparameter for Decision Tree


In [80]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [81]:
tree_clf = DecisionTreeClassifier(random_state=42)

In [82]:
param_grid = {"criterion": ["gini", "entropy"],
              "min_samples_split": range(1,10),
              "max_leaf_nodes":[5, 10, 15, 20]}

In [83]:
grid_search_tree = GridSearchCV(tree_clf, param_grid, cv=5, verbose=2, n_jobs=-1)

In [84]:
grid_search_tree.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
40 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 889, in fit
    super().fit(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 177, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _valida

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_leaf_nodes': [5, 10, 15, 20],
                         'min_samples_split': range(1, 10)},
             verbose=2)

In [85]:
grid_search_tree.best_params_

{'criterion': 'gini', 'max_leaf_nodes': 20, 'min_samples_split': 2}

In [86]:
best_tree_clf = grid_search_tree.best_estimator_

In [87]:
accuracy = best_tree_clf.score(X_test, y_test)

In [88]:
accuracy

0.8648

* The best tuned Decision Tree model has achieved around 87% accuracy.

## Question 4

### Load Dataset


In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml("mnist_784", version=1, as_frame=False)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


### Split into Train Set, Validation Set, and Test Set

In [2]:
X, y = mnist["data"], mnist["target"]
X_train, X_val, X_test = X[: 50000], X[50000:60000], X[60000:]
y_train, y_val, y_test = y[: 50000], y[50000:60000], y[60000:]

### Train Various Classifiers

In [3]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

rnf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
et_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)

In [5]:
for clf in (rnf_clf, et_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

RandomForestClassifier 0.968
ExtraTreesClassifier 0.9703


### Combine into an Ensemble

In [6]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(
    estimators=[('rf', rnf_clf), ('et', et_clf )],
    voting='soft')

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('et', ExtraTreesClassifier(random_state=42))],
                 voting='soft')

### Evaluate on the Validation Set

In [7]:
y_val_pred_ensemble = voting_clf.predict(X_val)
accuracy_val_ensemble = accuracy_score(y_val, y_val_pred_ensemble)
accuracy_val_ensemble

0.9749

In [8]:
y_val_pred_rf = rnf_clf.predict(X_val)
accuracy_val_rf = accuracy_score(y_val, y_val_pred_rf)
accuracy_val_rf

0.9736

In [9]:
y_val_pred_et = et_clf.predict(X_val)
accuracy_val_et = accuracy_score(y_val, y_val_pred_et)
accuracy_val_et

0.9743

* The ensemble classifier outperforms each individual classifier on the validation set.

### Evaluate on the Test Set

In [10]:
y_pred_ensemble = voting_clf.predict(X_test)
accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
accuracy_ensemble

0.9698

In [12]:
y_pred_rf = rnf_clf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
accuracy_rf

0.968

In [14]:
y_pred_et = et_clf.predict(X_test)
accuracy_et = accuracy_score(y_test, y_pred_et)
accuracy_et

0.9703

* For the test set, the ensemble outperforms Random Forest slightly, but Extra Trees performs slightly better than the ensemble one.